In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from mplsoccer import VerticalPitch
import os
from PIL import Image
import glob


In [ ]:

# 0. SET PARAMETERS
# ================

team_name = 'Belgium'
team_id = 43935
team_substitutions = [(18, 8), (10, 14), (23, 22), (16, 17)]  #[(18, 8), (10, 14), (23, 22), (16, 17), (15, 9)]


# team_name = 'Morocco'
# team_id = 43872
# team_substitutions = [(8, 18)]                 #[(15, 25), (2, 11), (17, 14), (19, 9), (8, 18)]


#siehe frames in tracking.csv
start_frame =    1558300   #1513390             #FRAMES ANPASSEN ############################################################
halftime_frame = 1513370 
end_frame =        1581100     #1560090

file = os.getcwd() + '/data/wyscout.csv'

# HELPER FUNCTIONS
# ================

def transform_coords(coords):
    y = coords[0] * 100
    x = (1 - coords[1]) * 100
    return x, y

def transform_tracking_coords(coords):
    y = (coords[0] + 5250) / (2*5250) * 100
    x = (1 - (coords[1] + 3400) / (2*3400)) * 100
    return x, y




# 1. Filter out frames we're not interested in (game is dead or it is out of the considered timeframe)
# ====================================================================================================

tracking_df = pd.read_csv('data/tracking.csv')

# select all rows where state is "Alive"
alive_entities = tracking_df[tracking_df['state'] == 'Alive']

# get the frames that contain alive entities
frames_with_alive_entities = alive_entities['frame'].unique()

# filter out these frames from the original dataframe
df_filtered = tracking_df[tracking_df['frame'].isin(frames_with_alive_entities)]\

# filter out frames that we are not interested in
df_filtered = df_filtered[df_filtered['frame'] >= start_frame]
df_filtered = df_filtered[df_filtered['frame'] <= end_frame]

# 2. Flip coordinates for second half of the match
# ================================================

df_filtered.loc[df_filtered['frame'] >= halftime_frame, 'x'] = - df_filtered.loc[df_filtered['frame'] >= halftime_frame, 'x']
df_filtered.loc[df_filtered['frame'] >= halftime_frame, 'y'] = - df_filtered.loc[df_filtered['frame'] >= halftime_frame, 'y']

# 3. Calculate the mean (x, y) coordinates of each player
# =======================================================

mean_coords = df_filtered.groupby(['team_id','player_jersey_number'])['x', 'y'].mean()

# 4. Draw plot
# ============

# draw pitch
pitch = VerticalPitch(pitch_color='w', line_color='k', pitch_type='wyscout')
fig, ax = pitch.draw(figsize=(16, 12.4))
ax.text(50, 101 , 'MINUTES 45-60',fontweight='bold', fontsize=30, color='green')         #ZEIT IM TITEL EINSTELLEN##############################

#IMPORT LOGOS----------------------------------------------------------------------------  LOGO ANPASSEN #######################################
#add and place logos

NED_logo = Image.open('wappen/belgien.png')
logo1_ax = fig.add_axes([0.2, 0.1, 0.25, 0.2], zorder=1)
logo1_ax.imshow(NED_logo)
logo1_ax.axis("off")

# SEN_logo = Image.open('wappen/morocco.png')
# logo2_ax = fig.add_axes([0.2, 0.73, 0.25, 0.2], zorder=1)
# logo2_ax.imshow(SEN_logo)
# logo2_ax.axis("off")
#--------------------------------------------------------------------------------

# get player location map
player_loc = {}
for player, entity in mean_coords.iterrows():
    if player[0] != team_id:
        continue
    x, y = transform_tracking_coords([entity['x'], entity['y']])
    player_loc[(player[0], int(player[1]))] = (x, y)

# draw substitution vectors and get player colors
player_color = {}
for o, i in team_substitutions:
    xo, yo = player_loc[(team_id, o)]
    player_color[(team_id, o)] =   'red' #'#fc6005'
    xi, yi = player_loc[(team_id, i)]
    player_color[(team_id, i)] =     'green' #'#09fc05'
    ax.arrow(xo, yo, xi - xo, yi - yo, head_width=0.05, head_length=0.1, fc='red', ec='black')

# draw players
for team, player in player_loc:
    x, y = player_loc[(team, player)]
    if (team, player) in player_color:
        color = player_color[team, player]
    else:
        # color = '#006233' #Morocco
        # color = '#EF3340' #Belgium
        color= 'black'

    circle = Ellipse((x, y), 4*1.05, 4*0.68, color=color, ec='black')
    ax.annotate(player, (x, y), color='white', weight='bold',
                 fontsize=10, ha='center', va='center')
    ax.add_patch(circle)


# save plot as svg
# fig.savefig(f'Graphics/AvgPos_Belgium/{team_name}_real_formations.svg', format='svg')
# fig.savefig(f'Graphics/AvgPos_{team_name}/{team_name}_real_formations4.png', format='png')          #DATEINAME ÄNDERN ####################################


# Create GIF from the images


In [395]:

# team_name = 'Morocco'
team_name = 'Belgium'

frame_folder= os.getcwd() + f"/Graphics/AvgPos_{team_name}"

frames=[]
imgs= glob.glob(frame_folder+"/*.png")
imgs.sort()


for i in imgs:
    new_frame= Image.open(i)
    frames.append(new_frame)

frames[0].save(frame_folder+f"/{team_name}_formation.gif", format="GIF", append_images=frames[1:], save_all=True, duration=1200, loop=0)






In the following illustration, the average position of each player is plotted and thus the actual formation during the game in 15 minutes intervals. If there are substitutions within an interval, the substituted player is marked in red and the new player in green.

## fist half

We can see that Morocco started the game in a 4-3-3 formation with Amrabat as CDM and Amallah and Ounahi as CM.
The only player to be in the opponents half is the striker En-Nesyri, all other players are in the own half. 
A very compact block can be seen, which indicates that Morocco started the game rather defensively, letting Belgium dictate the first minutes.
This changed in the following 15 minutes. The entire team stood higher and much wider, showing that they did not want to defend only. 
The last 15 minutes before halftime were similar as the beginning since almost all players were in their own half, but they were not that compact anymore. 
This shows that Belgium might have had much more possesion but Morocco still tried to attack instead of "parking the bus".


The first thing to notice in Belgiums formation in the beginning is the asymmetry. They played in a 4-2-3-1 formation with de Bruyne and Thorgan Hazard as wingers, while Eden Hazard was a CAM right behind Batshuayi. The asymmetrical shape comes about due to de Bruyne playing as a winger, since at his club he rather plays in the middle, mostly as a CAM, thats probably why he tends to move inwards, which in turn creates a lot of space on the right wing that Meunier could use and therefore he stood higher than left back Castagne.
The asymmetry could be observed in the whole first half.

In accordance to Morocco's rather defensive start, Belgium started more offensive, with most of the players in the opponents half. Between minutes 15 and 30, again, we could see that Morocco came in the game and Belgium stood a bit less high. Comparing both teams in that phase, we could conclude that it was a very competitve and balanced game. In the last 15 minutes before the break, Belgium stood higher and it appeared that they were having a lot of space on the wings but they were also closing their own space in the middle, which is where their superstars were, such that it was not too difficult for Morocco to defend.


## second half


The second half started rather balanced. No team seemed to have the upper hand in the staring minutes.

Between the minutes 60 and 75 there were four substitutions on Moroccos side. While two of which were position-specific, the other two disrupted the structure they had up to that moment. The attacking midfielder Amallah was substituted for left back Attiyat Allah, while Mazraoui moved inwards to play as a CDM next to Amrabat. Also Hakimi (right back) was substituted for midfielder  Sabiri. Their formation in this phase of the game seemed more attacking than before. This resulted in the first goal of the game in the 75th minute. In the last 15 minutes of the game the last substitution happened with defender El Yamiq replacing Ounahi. In the visualisation, it is easy to see that Morocco changed tactics and finished the game playing with five defenders and two rather defensive midfielders. The remaining players were the only ones to try to counter attack when possible. In the dying seconds, one counter attack resulted in the decisive 2-0.

In the 60th minute, there were two substitutions on Belgiums side. Since it was right between two slides, they were not plotted,but they did not change the formation or tactics. The only change was that de Bruyne changed sides and Mertens took over de Bruyne's side. 
In the following 15 minutes there were two position specific substitutions. 
This phase seemed very similar  to the starting of the second half. The game seemed very balanced. This changed drastically after the first goal.

Belgium had to throw in everything they had to equalize. Therefore, Lukaku replaced Meunier. We could see that they were much more attacking and standing much higher than before the goal, in accordance to Morocco's defensive tactics. In this part of the game there were only two trained defenders on the field (Alderweireld and Verthongen), that is why Morocco had a lot of space in Belgium's half for counter attacks. Even though Belgium were having lots of possesion and were in the opponents half, it appeared that they did not use he whole width of the field. 

